In [1]:
import pandas as pd
import numpy as np
import os
import re
import time
import threading
import subprocess
from glob import glob

from tqdm.notebook import tqdm
from rdkit import Chem
from rdkit.Chem import Descriptors, Descriptors3D, ChemicalFeatures, GraphDescriptors, Lipinski, rdchem
from rdkit.Chem.rdchem import Mol, Atom, Bond
from rdkit.Chem.rdmolfiles import SDMolSupplier, SDWriter
from rdkit.rdBase import BlockLogs
import datamol as dm
import multiprocessing

temp_path = './temp'
data_path = './data'

if not os.path.exists(temp_path):
    os.mkdir(temp_path)

In [2]:
xyz_filepath_list = list(glob(f'{data_path}/*.xyz'))

xyz_filepath_list.sort()
print('total xyz filepath # ', len(xyz_filepath_list))
xyz_filepath_list[0]

total xyz filepath #  133885


'./data/dsgdb9nsd_000001.xyz'

In [3]:
def check_missing_files() -> None:
    """
    Checks for missing .xyz QM9 files

    Return
        True | False: True, if all files are accounted for
    """
    for file in tqdm(xyz_filepath_list):
        if not os.path.exists(file):
            print('Retrieving QM9..')
            !wget https://figshare.com/ndownloader/files/3195389 -O ./data/data.bz2
            !tar -xfu ./data/data.bz2
            !rm ./data/*.bz2

    if not os.path.exists(f'{data_path}/edrug3d.sdf'):
        print('Retrieving edrug3d..')
        !wget https://chemoinfo.ipmc.cnrs.fr/TMP/tmp.33880/e-Drug3D_2056.sdf -O ./data/edrug3d.sdf

    return None

check_missing_files()

  0%|          | 0/133885 [00:00<?, ?it/s]

In [4]:
def in_ipython():
    try:
        return __IPYTHON__
    except NameError:
        return False

In [5]:
manager = multiprocessing.Manager()
molecules = manager.list()


def xyz2mol(m):
    blockedLogs = BlockLogs()

    with open(m) as file:
        lines = file.readlines()

        smiles = lines[len(lines) - 2].split()[0]
        smiles = dm.standardize_smiles(smiles)

        mol = dm.to_mol(smiles, add_hs=True, sanitize=True, ordered=True)
        mol = dm.fix_mol(mol)
        dm.align.compute_2d_coords(mol)
        # dm.conformers.generate(mol)

        molecules.append(mol)


def e3d2mol():
    blockedLogs = BlockLogs()

    mols = dm.read_sdf(f'{data_path}/edrug3d.sdf', remove_hs=False)
    molecules.extend(mols)


# Not working yet
# e3d2mol()

dm.parallelized(xyz2mol,
                xyz_filepath_list,
                n_jobs=-1,
                progress=True,
                arg_type='arg',
                total=len(xyz_filepath_list))

print('Done')

  0%|          | 0/133885 [00:00<?, ?it/s]

Done


In [6]:
len(molecules)

133885

In [7]:
df_list = []
main_df = None
line_split_re = re.compile(r'\s+')


def preprocess():
    for mol in tqdm(molecules):
        if mol is None:
            continue

        # mol = Chem.AddHs(mol)

        # Mol.Compute2DCoords(mol)

        df_out = None

        with open(temp_path + '/mol.sdf', 'w') as file:
            Chem.MolToPDBFile(mol, temp_path + '/mol.sdf')

        # Run antechamber and divert output to a file (temporary)
        subprocess.getoutput(
            f'cd {temp_path} && antechamber -i mol.sdf -fi pdb -o mol.ac -fo ac -at gaff2 -pf y'
        )

        # Info = {}

        # rdInfo['hvyAtCnt'] = Lipinski.HeavyAtomCount(mol)
        # rdInfo['nhohC'] = Lipinski.NHOHCount(mol)
        # rdInfo['noC'] = Lipinski.NOCount(mol)
        # rdInfo['aliCC'] = Lipinski.NumAliphaticCarbocycles(mol)
        # rdInfo['aliHC'] = Lipinski.NumAliphaticHeterocycles(mol)
        # rdInfo['aliR'] = Lipinski.NumAliphaticRings(mol)
        # rdInfo['aroCC'] = Lipinski.NumAromaticCarbocycles(mol)
        # rdInfo['aroR'] = Lipinski.NumAromaticRings(mol)
        # rdInfo['HA'] = Lipinski.NumHAcceptors(mol)
        # rdInfo['HD'] = Lipinski.NumHDonors(mol)
        # rdInfo['HAt'] = Lipinski.NumHeteroatoms(mol)
        # rdInfo['rotatableBonds'] = Lipinski.NumRotatableBonds(mol)
        # rdInfo['satCC'] = Lipinski.NumSaturatedCarbocycles(mol)
        # rdInfo['satHC'] = Lipinski.NumSaturatedHeterocycles(mol)
        # rdInfo['satR'] = Lipinski.NumSaturatedRings(mol)
        # rdInfo['rC'] = Lipinski.RingCount(mol)
        # rdInfo['molWt'] = Descriptors.MolWt(mol)
        # rdInfo['hvyAtMolWt'] = Descriptors.HeavyAtomMolWt(mol)
        # rdInfo['molMaxPC'] = Descriptors.MaxPartialCharge(mol)
        # rdInfo['molMinPC'] = Descriptors.MinPartialCharge(mol)
        # rdInfo['molRadElec'] = Descriptors.NumRadicalElectrons(mol)
        # rdInfo['molValElec'] = Descriptors.NumValenceElectrons(mol)
        # rdInfo['asph'] = Descriptors3D.Asphericity(mol)
        # rdInfo['ecc'] = Descriptors3D.Eccentricity(mol)
        # rdInfo['isf'] = Descriptors3D.InertialShapeFactor(mol)
        # rdInfo['npr1'] = Descriptors3D.NPR1(mol)
        # rdInfo['npr2'] = Descriptors3D.NPR2(mol)
        # rdInfo['pmi'] = Descriptors3D.PMI1(mol)
        # rdInfo['rog'] = Descriptors3D.RadiusOfGyration(mol)
        # rdInfo['j'] = GraphDescriptors.BalabanJ(mol)
        # rdInfo['ct'] = GraphDescriptors.BertzCT(mol)

        rdInfo = dm.descriptors.compute_many_descriptors(mol, {
            'hvyAtCnt': 'HeavyAtomCount',
            'nhohC': 'NHOHCount',
            'noC': 'NOCount',
            'aliCC': 'NumAliphaticCarbocycles',
            'aliHC': 'NumAliphaticHeterocycles',
            'aliR': 'NumAliphaticRings',
            'aroCC': 'NumAromaticCarbocycles',
            'aroR': 'NumAromaticRings',
            'HA': 'NumHAcceptors',
            'HD': 'NumHDonors',
            'HAt': 'NumHeteroatoms',
            'rotatableBonds': 'NumRotatableBonds',
            'satCC': 'NumSaturatedCarbocycles',
            'satHC': 'NumSaturatedHeterocycles',
            'satR': 'NumSaturatedRings',
            'rC': 'RingCount',
            'molWt': 'MolWt',
            'hvyAtMolWt': 'HeavyAtomMolWt',
            'molMaxPC': 'MaxPartialCharge',
            'molMinPC': 'MinPartialCharge',
            'molRadElec': 'NumRadicalElectrons',
            'molValElec': 'NumValenceElectrons',
        })

        with open(temp_path + '/mol.ac') as file:
            lines = [a for a in file.readlines() if 'ATOM' in a]

            for atom in Mol.GetAtoms(mol):
                out = {}

                idx = Atom.GetIdx(atom)

                line = lines[idx]

                a_split = line_split_re.split(line.strip())
                atom_name = a_split[2]
                if len(a_split) > 9:
                    atom_type = a_split[9]
                else:
                    atom_type = a_split[8]

                out['type'] = atom_type
                out['name'] = Atom.GetSymbol(atom)

                # print(f'RDKit atom: {Atom.GetSymbol(atom)}')
                # print(f'antechamber atom: {atom_name}')

                bonds = {
                    'C': 0,
                    'H': 0,
                    'N': 0,
                    'O': 0,
                    'S': 0,
                    'F': 0,
                    'Cl': 0,
                    'Br': 0,
                    'I': 0,
                    'Other': 0
                }

                bond_types = {
                    'SINGLE': 0,
                    'DOUBLE': 0,
                    'TRIPLE': 0,
                    'OTHER': 0
                }

                args = {
                    'depDeg': Atom.GetDegree(atom),
                    'explVal': Atom.GetExplicitValence(atom),
                    'forC': Atom.GetFormalCharge(atom),
                    'implV': Atom.GetImplicitValence(atom),
                    'isA': int(Atom.GetIsAromatic(atom)),
                    'explH': Atom.GetNumExplicitHs(atom),
                    'implH': Atom.GetNumImplicitHs(atom),
                    'radEl': Atom.GetNumRadicalElectrons(atom),
                    'totDeg': Atom.GetTotalDegree(atom),
                    'totH': Atom.GetTotalNumHs(atom),
                    'totV': Atom.GetTotalValence(atom),
                    'isR': int(Atom.IsInRing(atom)),
                }

                for bond in Atom.GetBonds(atom):
                    bond_name = Atom.GetSymbol(Bond.GetEndAtom(bond))
                    bond_type = Bond.GetBondType(bond)

                    if bond_name in bonds.keys():
                        bonds[bond_name] += 1
                    else:
                        bonds['Other'] += 1

                    if bond_type in bond_types.keys():
                        bond_types[bond_type] += 1
                    else:
                        bond_types['OTHER'] += 1

                out.update(bonds)
                out.update(bond_types)
                out.update(rdInfo)
                out.update(args)

                # Append molecule info to every atom
                atom_df = pd.DataFrame(out, index=[0])

                if df_out is None:
                    df_out = pd.DataFrame(columns=atom_df.columns)

                df_out = pd.concat([df_out, atom_df], ignore_index=True)

        # Clean up remaining files
        try:
            os.remove(temp_path + '/mol.ac')
            os.remove(temp_path + '/mol.sdf')
        except IOError:
            print('Something went wrong.')

        df_list.append(df_out)

In [8]:
def main():
    preprocess()

    # Concatenate dfs in the list
    main_df = pd.concat(df_list, ignore_index=True)
    main_df.to_csv(f'{data_path}/data.csv', index=False)

In [ ]:
if __name__ == "__main__":
    if not in_ipython():
        root_dir = os.path.dirname(os.path.realpath(__file__))

        main()
    else:
        main()

  0%|          | 0/133885 [00:00<?, ?it/s]